In [1]:
import pandas as pd
import numpy as np

In [2]:
def load_data(filename):
    data = pd.read_csv(filename,sep=' ').reset_index()
    data = data.rename(columns={data.columns[0]:'x', data.columns[1]:'y'})
    return data

In [3]:
def dist2(p1,p2):
    return (p1['x']-p2['x'])**2 + (p1['y']-p2['y'])**2

In [4]:
def dist(p1,p2):
    return np.sqrt(dist2(p1,p2))

In [5]:
def closest_point(visited_point, data):
    p0 = data.loc[visited_point[-1]]
    return data.loc[~data.index.isin(visited_point)].apply(
        lambda p: dist2(p,p0),
        axis=1
    ).argmin()

In [40]:
def tsp_simple(data):
    sol = [data.index[0]]
    while len(sol) < len(data):
        sol += [closest_point(sol,data)]
    return sol

In [57]:
def tsp_simple_fast(data,closest_matrix):
    sol = [data.index[0]]
    while len(sol) < len(data):
        last = sol[-1]
        sol += [closest_matrix[last][~closest_matrix[last].isin(sol)].values[0]]
    return sol

In [8]:
def objective(sol, data):
    return sum([dist(data.loc[p1],data.loc[p2]) for p1, p2 in zip(sol,sol[1:]+[sol[0]])])  

In [9]:
def objective2_vec(sol, data):
    return [dist2(data.loc[p1],data.loc[p2]) for p1, p2 in zip(sol,sol[1:]+[sol[0]])] 

In [10]:
def objective2(sol, data):
    return sum(objective2_vec(sol, data))  

In [11]:
def save_sol(sol, data, filename):
    obj = objective(sol,data)
    output_data = str(obj) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, sol))
    f = open(filename.replace('data','out'),'w')
    f.write(output_data)
    f.close()

In [12]:
def shift(seq, n):
    return seq[n:]+seq[:n]

In [13]:
def swap_tsp_2(sol, p1, p2):
    pos1 = sol.index(p1)
    shift_sol = shift(sol,pos1)
    pos2 = shift_sol.index(p2)
    res_sol = shift_sol[:pos2]+shift_sol[pos2:][::-1]
    return shift(res_sol,len(sol)-pos1)

In [14]:
def objective_swap(sol, obj, p1 ,p2, data):
    pos1 = sol.index(p1)
    pos2 = sol.index(p2)
    p1_next = sol[pos1-1]
    p2_next = sol[pos2-1]
    dp1 = data.loc[p1]
    dp2 = data.loc[p2]
    dp1_next = data.loc[p1_next]
    dp2_next = data.loc[p2_next]
    return obj - dist2(dp1,dp1_next) - dist2(dp2,dp2_next) + dist2(dp1,dp2) + dist2(dp1_next,dp2_next)

In [15]:
def objective_vec_swap(sol, obj_vec, p1 ,p2, data):
    pos1 = sol.index(p1)
    shift_sol = shift(sol,pos1)
    shift_obj_vec = shift(obj_vec,pos1)
    pos2 = shift_sol.index(p2)
    res_obj_vec = shift_obj_vec[:pos2]+ (shift_obj_vec[pos2:][::-1][1:]+[0])
    p1_next = sol[pos1-1]
    p2_next = shift_sol[pos2-1]
    dp1 = data.loc[p1]
    dp2 = data.loc[p2]
    dp1_next = data.loc[p1_next]
    dp2_next = data.loc[p2_next]
    res_obj_vec[pos2-1]=dist2(dp1_next,dp2_next)
    res_obj_vec[-1]=dist2(dp1,dp2)
    return shift(res_obj_vec,len(sol)-pos1)

In [16]:
def local_search(
    data,
    max_trials,
    start_sol,
    start_obj_vec,
    closest_matrix,
    first_closest=None):
    
    size = len(start_sol)
    cur_sol = start_sol
    cur_obj_vec = start_obj_vec
    cur_obj = sum(cur_obj_vec)
    opt_sol = start_sol
    opt_obj_vec = cur_obj_vec
    opt_obj = cur_obj
    
    it = 0
    while (it < max_trials):
        order_points = pd.Series(cur_obj_vec).sort_values(ascending=False).index
        found = False
        max_trials_reached = False
        for p1 in order_points:
            for idx2 in closest_matrix[p1].index[1:first_closest if first_closest is not None else size]:
                if it >= max_trials:
                    max_trials_reached = True
                    break
                else:
                    it+=1
                p2 = closest_matrix[p1][idx2]
                #print 'try (%d %d)'%(p1,p2)
                new_obj = objective_swap(cur_sol, cur_obj, p1, p2, data)
                if new_obj < opt_obj:
                    cur_obj_vec = objective_vec_swap(cur_sol,cur_obj_vec,p1,p2,data)
                    cur_obj = sum(cur_obj_vec)
                    cur_sol = swap_tsp_2(cur_sol,p1,p2)
                    #print 'it %d order %d closest rank %d swap (%d %d) obj2 %f obj: %f'%(it,order_points.get_loc(p1),idx2, p1,p2,cur_obj,sum(map(np.sqrt,cur_obj_vec)))
                    opt_sol = cur_sol
                    opt_obj_vec = cur_obj_vec
                    opt_obj = sum(cur_obj_vec)
                    found = True
                    break
            if found or max_trials_reached:
                break
    return sum(map(np.sqrt,opt_obj_vec)), opt_sol 

In [17]:
from datetime import datetime

In [18]:
def dist_matrix_calc(data):
    data_x = np.repeat(data['x'].values,len(data)).reshape((len(data),len(data)))
    data_y = np.repeat(data['y'].values,len(data)).reshape((len(data),len(data)))
    return pd.DataFrame((data_x-data_x.T)*(data_x-data_x.T)+(data_y-data_y.T)*(data_y-data_y.T))

In [27]:
def closest_matrix_calc_step(data, start_idx, end_idx, first_closest):
    data_x = data.x.values
    data_y = data.y.values
    data_x_1 = np.repeat(data_x,end_idx-start_idx).reshape(len(data),end_idx-start_idx)
    data_x_2 = np.repeat(data_x[start_idx:end_idx],len(data)).reshape((end_idx-start_idx,len(data))).T
    data_y_1 = np.repeat(data_y,end_idx-start_idx).reshape(len(data),end_idx-start_idx)
    data_y_2 = np.repeat(data_y[start_idx:end_idx],len(data)).reshape((end_idx-start_idx,len(data))).T
    dist_matrix = pd.DataFrame((data_x_2-data_x_1)*(data_x_2-data_x_1)
                               +(data_y_2-data_y_1)*(data_y_2-data_y_1),
                              columns=range(start_idx,end_idx))
    return pd.DataFrame([dist_matrix[c].nsmallest(first_closest).index for c in dist_matrix.columns])

In [20]:
def closest_matrix_calc(data, first_closest, step=100):
    steps = range(0,len(data),step)+[len(data)]
    return pd.concat([closest_matrix_calc_step(data,start_idx,end_idx,first_closest) 
               for (start_idx, end_idx) in zip(steps[:-1],steps[1:])],axis=1)

In [21]:
def solve_tsp(filename,max_trials, first_closest, closest_matrix=None, calc_first_closest=1000, step=1000):
    data = load_data(filename)
    start_time = datetime.now()
    if closest_matrix is None:
        closest_matrix = closest_matrix_calc(data,calc_first_closest,step)
    closest_matrix_time = datetime.now()
    print 'closest_matrix time : %d'%((closest_matrix_time-start_time).seconds)
    sol_step0 = tsp_simple_fast(data,closest_matrix)
    start_sol_time = datetime.now()
    print 'start sol time : %d'%((start_sol_time-closest_matrix_time).seconds)
    obj_vec = objective2_vec(sol_step0,data)
    opt_obj, opt_sol = local_search(data,max_trials,sol_step0,obj_vec,closest_matrix,first_closest)
    start_local_search_time = datetime.now()
    print 'local_search time : %d'%((start_local_search_time-start_sol_time).seconds)
    return opt_obj, opt_sol

In [22]:
PbFilename1 = r'./data/tsp_51_1'
PbFilename2 = r'./data/tsp_100_3'
PbFilename3 = r'./data/tsp_200_2'
PbFilename4 = r'./data/tsp_574_1'
PbFilename5 = r'./data/tsp_1889_1'
PbFilename6 = r'./data/tsp_33810_1'

In [71]:
data = load_data(PbFilename2)

In [53]:
%time closest_matrix = closest_matrix_calc(data,len(data),len(data))

Wall time: 15.6 s


In [72]:
opt_obj, opt_sol = solve_tsp(PbFilename2,10000,10,None,len(data),len(data))
save_sol(opt_sol,data,PbFilename2)
print opt_obj

closest_matrix time : 0
start sol time : 0
local_search time : 35
35014.0184854
